In [1]:
import pennylane as qml
from pennylane import numpy as np

## V.2.1

In [ ]:
n_bits = 2
dev = qml.device("default.qubit", wires=n_bits, shots=None)
np.random.seed(41)
def build_hamiltonian():
    """Build the Hamiltonian.

    Returns:
        (qml.Hamiltonian): Hamiltonian operator.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    return qml.dot([1., -1.], [qml.Z(0) @ qml.Z(1), qml.X(0) @ qml.X(1)])

def run_vqe():
    """Run VQE algorithm with initial parameters defined by the user.

    Returns:
        array(float): Ground state energy of the Hamiltonian.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    shape = qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=n_bits)
    weights = np.random.random(size=shape)
    observable = build_hamiltonian()
    weights = optimizer(observable, weights)
    ground_state_energy = cost_function(observable, weights)
    return ground_state_energy


## V.2.2a

In [2]:
# Import the H2 molecule dataset
dataset = qml.data.load('qchem', folder_path="/tmp", molname="H2")[0]
# Define Hamiltonian and qubits
H, qubits = dataset.hamiltonian, len(dataset.hamiltonian.wires)
# The Hartree-Fock State
hf = dataset.hf_state
# Define the single and double excitations
singles, doubles = qml.qchem.excitations(electrons=2, orbitals=qubits)
num_params = len(singles) + len(doubles)

print(num_params)

3


In [3]:
print(singles, doubles)

[[0, 2], [1, 3]] [[0, 1, 2, 3]]


In [4]:
# Import the H2 molecule dataset
dataset = qml.data.load('qchem', folder_path="/tmp", molname="H2")[0]
# Define Hamiltonian and qubits
H, qubits = dataset.hamiltonian, len(dataset.hamiltonian.wires)
# The Hartree-Fock State
hf = dataset.hf_state
# Define the single and double excitations
singles, doubles = qml.qchem.excitations(electrons=2, orbitals=qubits)
num_params = len(singles) + len(doubles)

def hf_ansatz(params):
    """Build the Hartree-Fock ansatz.
    
    Args:
        params (np.array): An array with the angles of the single and double excitations.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    qml.AllSinglesDoubles(params, wires=range(qubits), hf_state=hf, singles=singles, doubles=doubles)

dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev, interface="autograd")
def cost_hf(params):
    """Build the cost function using the Hartree-Fock ansatz.
    Args:
        params (np.array): An array with the angles of the single and double excitations.

    Returns:
        (np.tensor): Energy of the Hamiltonian.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    hf_ansatz(params)
    return qml.expval(H)

## V2.2b

In [7]:
# Import the H2 molecule dataset
dataset = qml.data.load('qchem', folder_path="/tmp", molname="H2")[0]
# Define Hamiltonian and qubits
H, qubits = dataset.hamiltonian, len(dataset.hamiltonian.wires)
# The Hartree-Fock State
hf = dataset.hf_state
# Define the single and double excitations
singles, doubles = qml.qchem.excitations(electrons=2, orbitals=qubits)
num_params = len(singles) + len(doubles)

def run_VQE_hf():
    """Executes the VQE optimizing the parameters of the Hartree-Fock ansatz.

    Returns:
        (np.array): an array with the optimized trainable parameters.
        (float): the ground state energy
    """
    ##################
    # YOUR CODE HERE #
    ##################
    params = np.random.random(size=(num_params,))
    optimizer = qml.AdamOptimizer()
    prev, params = params, optimizer.step(cost_hf, params)
    while not np.allclose(prev, params):
        prev, params = params, optimizer.step(cost_hf, params)
    return params, cost_hf(params)

run_VQE_hf()

tensor([1.45990979e-07, 1.96591299e-08, 2.26381722e-01], requires_grad=True)